In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

In [2]:
Customer = namedtuple('Customer', ['name', 'fidelity'])

## ”策略“模式

In [3]:
# 商品类

class LineItem:
    
    def __init__(self, product_name, quantity, price):
        self.product = product_name
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity

In [6]:
# 促销策略类

class Promotion(ABC):
    @abstractmethod
    def discount(self, order):
        pass
    
    
class FidelityPromo(Promotion):
    """积分为1000或者以上时，提供5%折扣"""
    
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
    

class BulkItemPromo(Promotion):
    """单个商品为20个或者以上时，对满足条件的商品提供10%折扣"""
    
    discount = 0
    def discount(self, order):
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount
    
    
class LargeOrderPromo(Promotion):
    '''订单中不同商品的种类达到10个或以上时，对所有商品提供7%折扣'''
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}  # set
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

In [8]:
# 汇总类

class Order:
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)  # cart's type: class LineItem
        self.promotion = promotion  # promotion's type: class Promotion
        
    def total(self):
        if not hasattr(self, '_total'):
            self._total = sum(item.total() for item in self.cart)
        return self._total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)  # 将self作为参数
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [9]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

cart = [LineItem('banana', 4, 0.5), 
       LineItem('apple', 10, 1.5), 
       LineItem('watermellon', 5, 5.0)]

Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

#### 上面的栗子中，每个具体策略都是一个类，而且都只定义了一个方法， 即 discount。此外，策略实例没有状态（没有实例属性）。它们看起来像是普通的函数——的确如此。下面把具体策略换成了简单的函数，而且去掉了 Promo 抽象类。

In [10]:
# 商品类

class LineItem:
    
    def __init__(self, product_name, quantity, price):
        self.product = product_name
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity

In [11]:
def fidelity_promo(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

In [12]:
# 汇总类

class Order:
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)  # cart's type: class LineItem
        self.promotion = promotion  # promotion's type: class Promotion
        
    def total(self):
        if not hasattr(self, '_total'):
            self._total = sum(item.total() for item in self.cart)
        return self._total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)  # 将self作为参数
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [18]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

cart = [LineItem('banana', 4, 0.5), 
       LineItem('apple', 10, 1.5), 
       LineItem('watermellon', 5, 5.0)]

banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [28]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    return max(promo(order) for promo in promos)

In [29]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

## globals

In [30]:
promos = [globals()[name] for name in globals() 
          if name.endswith('_promo') and name != 'best_promo']

In [31]:
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

In [33]:
import inspect

In [34]:
promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction())]

NameError: name 'promotions' is not defined

## "命令"模式

In [ ]:
class MacroCommand:
    
    def __init__(self, commands):
        self.commands = list(commands)
        
    def __call__(self):
        for command in self.commands:
            command()